This notebook contains the bulk of my 3rd place solution. This notebook uses some basic default XGBoost parameters, but for my final solution I used an average of 17 different tuned hyperparameters (improvement of ~.55 WMAE). For more details on the rest of my solution, check out my discussion post at https://www.kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2/discussion/563064

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor, DMatrix
import shap
import lightgbm as lgb
shap.initjs()

In [ ]:
inventory = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv').drop(['warehouse','product_unique_id'],axis=1)
inventory.head()

One thing I found very fruitful was extracting the word part of the 'name' field (in my code this I call this the 'common_name' and using it as a grouping column for various aggregations of other features.

Generally speaking, I didn't have a lot of success trying to extract additional features from the calendar. All I ended up grabbing were binary indicators of holidays, the day before a holiday, and the day after a holiday.

In [ ]:
def fe_date(df):
    df['year'] = df['date'].dt.year
    df['day_of_week'] = df['date'].dt.dayofweek
    df['days_since_2020'] = (df['date'] - pd.to_datetime('2020-01-01')).dt.days.astype('int')
    df['day_of_year'] = df['date'].dt.dayofyear
    df['cos_day'] = np.cos(df['day_of_year']*2*np.pi/365)
    df['sin_day'] = np.sin(df['day_of_year']*2*np.pi/365)

def fe_other(df):
    discount_cols = ['type_0_discount','type_1_discount','type_2_discount','type_3_discount','type_4_discount','type_5_discount','type_6_discount']
    df[discount_cols] = df[discount_cols].clip(0)
    df['max_discount'] = df[['type_0_discount','type_1_discount','type_2_discount','type_3_discount','type_4_discount','type_5_discount']].max(axis=1)
    
    # Given that we're using XGBoost, which is in theory invariant to monotonic transformations of features, this transformation in isolation doesn't really do anything. I mainly did it because it made the shap plot look more linear. However, I think it did make further feature engineering that used price more effective.
    df['sell_price_main'] = np.log(df['sell_price_main']) 

    df['common_name'] = df['name'].apply(lambda x: x[:x.find('_')])
    df['CN_total_products'] = df.groupby(['date','warehouse','common_name'])['unique_id'].transform('nunique')
    df['CN_discount_avg'] = df.groupby(['date','warehouse','common_name'])['max_discount'].transform('mean')
    df['CN_WH'] = df['common_name'] + '_' + df['warehouse']
    df['name_num_warehouses'] = df.groupby(['date','name'])['unique_id'].transform('nunique')

def fe_combined(df):
    df['num_sales_days_28D'] = pd.MultiIndex.from_frame(df[['unique_id','date']]).map(df.sort_values('date').groupby('unique_id').rolling(
        window='28D', on='date', closed='left')['date'].count().fillna(0))

    # This 'price_detrended' feature was one I found pretty late into the game, but I think it helped out a lot. I was trying to make a feature that captured whether an item was cheap or expensive relative to its usual price, which is what 'price_scaled' represents. What I found was that the prices of things generally increase over time. So I removed that time-based trend to construct price_detrended, and that proved very effective.
    mean_prices = df.groupby(df['unique_id'])['sell_price_main'].mean()
    std_prices = df.groupby(df['unique_id'])['sell_price_main'].std()
    df['price_scaled'] = np.where(df['unique_id'].map(std_prices) == 0, 0, 
                                  (df['sell_price_main'] - df['unique_id'].map(mean_prices))/df['unique_id'].map(std_prices))
    df['price_detrended'] = df['price_scaled'] - df.groupby(['days_since_2020','warehouse'])['price_scaled'].transform('mean')
    df.drop('price_scaled',axis=1,inplace=True)

    warehouse_stats = df.groupby(['date','warehouse'])['total_orders'].median().rename('med_total_orders').reset_index().sort_values('date')
    warehouse_stats['ewmean_orders_56'] = warehouse_stats.groupby('warehouse')['med_total_orders'].transform(lambda x:x.ewm(alpha=1/56).mean())
    df['mean_orders_14d'] = pd.MultiIndex.from_frame(df[['warehouse','date']]).map(
        warehouse_stats.groupby('warehouse').rolling(on='date',window='14D')['med_total_orders'].mean())
    df['ewmean_orders_56'] = pd.MultiIndex.from_frame(df[['warehouse','date']]).map(
        warehouse_stats.set_index(['warehouse','date'])['ewmean_orders_56'])
    return df

In [ ]:
calendar = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv', parse_dates=['date'])
calendar.loc[calendar['holiday_name'].isna(), 'holiday'] = 0 # V3
calendar['last_holiday_date'] = calendar['date']
calendar['next_holiday_date'] = calendar['date']
calendar.loc[calendar['holiday'] == 0, ['last_holiday_date','next_holiday_date']] = np.nan
calendar['last_holiday_date'] = calendar.sort_values('date').groupby('warehouse')['last_holiday_date'].ffill()
calendar['next_holiday_date'] = calendar.sort_values('date').groupby('warehouse')['next_holiday_date'].bfill()
calendar['days_since_last_holiday'] = ((calendar['date'] - calendar['last_holiday_date']).dt.days)
calendar['days_to_next_holiday'] = ((calendar['next_holiday_date'] - calendar['date']).dt.days)
calendar['day_before_holiday'] = calendar['days_to_next_holiday'] == 1
calendar['day_after_holiday'] = calendar['days_since_last_holiday'] == 1
calendar.drop(['last_holiday_date','next_holiday_date'],axis=1,inplace=True)
calendar.drop(['days_since_last_holiday','days_to_next_holiday'],axis=1,inplace=True)
calendar.drop(['shops_closed','winter_school_holidays','school_holidays','holiday_name'],axis=1,inplace=True)

In [ ]:
train = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv', parse_dates=['date'])
train['id'] = train['unique_id'].astype('str') + '_' + train['date'].astype('str')
train.set_index('id',inplace=True)
train = train[~train['sales'].isna()]
train = train.reset_index().merge(inventory, on='unique_id').set_index('id').loc[train.index]
train = train.reset_index().merge(calendar, on=['date','warehouse']).set_index('id').loc[train.index]
fe_date(train)
fe_other(train)

all_data = train
all_data = fe_combined(all_data)

train = all_data[all_data['date'] < pd.to_datetime('2024-05-20')]
test = all_data[all_data['date'] >= pd.to_datetime('2024-05-20')]
test_sales = test['sales']
test = test.drop(columns=['sales', 'availability'], axis=1)

In [ ]:
X_train = train.drop('sales',axis=1)
y_train = train['sales']
train_availability = X_train['availability']
X_train.drop('availability',inplace=True,axis=1)
weights = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv').set_index('unique_id')
X_train_weights = X_train['unique_id'].map(weights['weight'])

In [ ]:
cat_cols = ['unique_id'] + list(X_train.columns[X_train.dtypes == 'object'])
all_data = pd.concat([X_train, test])
add_cols = ['last_sales_ema005','CN_sales_sum','last_sales_zs']

# Here there are a few additional features engineered from historical sales data. These are done separately from the rest of my feature engineering because when I go to test model performance on a time-based holdout validation set, I need to make sure these features aren't using sales data from that validation set.
train_cp = train.groupby('unique_id')['date'].apply(lambda s: pd.date_range(s.min(), test.date.max())).explode().reset_index()
train_cp = train_cp.merge(
    pd.concat([train[['unique_id','date','sales','warehouse',]], 
               test[['unique_id','date','warehouse']]]),
    on=['unique_id','date'],how='left')
train_cp = train_cp.merge(inventory, left_on='unique_id', right_index=True)
train_cp['common_name'] = train_cp['name'].apply(lambda x: x[:x.find('_')])
train_cp.sort_values('date',inplace=True)
train_cp['last_sales_ema005'] = train_cp.groupby(['unique_id'])['sales'].transform(lambda x: x.shift(1).ewm(alpha=.005).mean()).fillna(0)
train_cp['CN_sales_sum'] = train_cp.groupby(['common_name','warehouse','date'])['last_sales_ema005'].transform('sum')
all_data = all_data.merge(train_cp.set_index(['unique_id','date'])[[
    'last_sales_ema005','CN_sales_sum'
]], left_on=['unique_id','date'],right_index=True,how='left')
sales_stats = train_cp.groupby(['common_name','warehouse'])['sales'].agg(['mean','std'])
all_data['last_sales_zs'] = (all_data['last_sales_ema005'] - pd.MultiIndex.from_frame(all_data[['common_name','warehouse']]).map(
    sales_stats['mean']))/ pd.MultiIndex.from_frame(all_data[['common_name','warehouse']]).map(sales_stats['std'])

# Cutting all data prior to 2022 seems to help. This could be due to COVID effects, and also the fact that there is little data from the Germany warehouses before 2022.
X_train = X_train[X_train['date'] >= '2022-01-01']
y_train = y_train.loc[X_train.index]
X_train_weights = X_train_weights.loc[X_train.index]

X_train[add_cols] = all_data[add_cols]
test[add_cols] = all_data[add_cols]
all_data[cat_cols] = all_data[cat_cols].astype('str').astype('category')

In [ ]:
# # Save all data for experiments
# import os
# SAVE_DIRECTORY = '/kaggle/working/dfs/'
# os.makedirs(SAVE_DIRECTORY, exist_ok=True)

# X_train.to_csv(SAVE_DIRECTORY + 'X_train.csv', index=False)
# y_train.to_csv(SAVE_DIRECTORY + 'y_train.csv', index=False)
# test.to_csv(SAVE_DIRECTORY + 'test.csv', index=False)
# test_sales.to_csv(SAVE_DIRECTORY + 'test_sales.csv', index=False)

## Trying own simple model - LGBM

In [ ]:
# DF_DIRECTORY = '/kaggle/input/3rd-place-dfs/'
# X_train = pd.read_csv(DF_DIRECTORY + 'X_train.csv')
# y_train = pd.read_csv(DF_DIRECTORY + 'y_train.csv')
# test = pd.read_csv(DF_DIRECTORY + 'test.csv')
# test_sales = pd.read_csv(DF_DIRECTORY + 'test_sales.csv')

In [ ]:
np.setdiff1d(weights['unique_id'], X_train['unique_id'])

In [ ]:
weights = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv')\
.set_index('unique_id')

In [ ]:
train_weight = X_train['unique_id'].map(weights['weight'])
test_weight = test['unique_id'].map(weights['weight'])

In [ ]:
object_cols = X_train.select_dtypes(include=['object']).columns
X_train[object_cols] = X_train[object_cols].astype('category')
test[object_cols] = test[object_cols].astype('category')

special = 'CN_WH'
X_train[special], _ = X_train[special].factorize()
test[special], _ = test[special].factorize()

In [ ]:
def custom_weighted_squared_mae(preds, dtrain):
    # Retrieve true labels
    labels = dtrain.get_label()
    
    # Retrieve weights; if not provided, default to an array of ones
    weights = dtrain.get_weight()
    if weights is None or len(weights) == 0:
        weights = np.ones_like(labels)
    
    # Square both predictions and labels
    squared_preds = preds ** 2
    squared_labels = labels ** 2
    
    # Compute the weighted mean absolute error
    weighted_mae = np.sum(np.abs(squared_preds - squared_labels) * weights) / np.sum(weights)
    
    # Return a tuple: (metric_name, metric_value, is_higher_better)
    return 'weighted_squared_mae', weighted_mae, False


In [ ]:
%%time

# Define features (exclude target and date; adjust if you need to drop additional columns)
features = [col for col in X_train.columns if col not in ['date', 'name']]

# Create LightGBM Datasets
lgb_train = lgb.Dataset(X_train[features], label=np.sqrt(y_train), weight=train_weight)
lgb_valid = lgb.Dataset(test[features], label=np.sqrt(test_sales), reference=lgb_train, weight=test_weight)

# Define LightGBM parameters (you can adjust these)
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'verbosity': -1,
    'seed': 40,
    'learning_rate': 0.1,
    'verbosity': -1,
    'device': 'gpu', 
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'max_bins': 255
}

# Train the model with early stopping
model = lgb.train(
    params,
    lgb_train,
    num_boost_round=5000,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=100)
    ],
    feval=custom_weighted_squared_mae
)

In [ ]:
%%time
train_preds = model.predict(X_train[features])
test_preds = model.predict(test[features])

In [ ]:
train_mae = mean_absolute_error(train_preds**2, y_train, sample_weight = train_weight)
test_mae = mean_absolute_error(test_preds**2, test_sales, sample_weight = test_weight)
print(train_mae)
print(test_mae)

In [ ]:
pd.Series(train_weight).isna().sum()

In [ ]:
lgb.plot_importance(model)

In [ ]:
explainer = shap.TreeExplainer(model)

X_subset = X_train[features].sample(10000)
shap_values = explainer.shap_values(X_subset)

shap.summary_plot(shap_values, X_subset, plot_type="dot")
plt.show()

In [ ]:
shap.dependence_plot('total_orders', shap_values, X_subset)

## XGboost model

In [ ]:
def custom_weighted_squared_mae(preds, dtrain):
    # Retrieve true labels
    labels = dtrain.get_label()
    
    # Retrieve weights; if not provided, default to an array of ones
    weights = dtrain.get_weight()
    if weights is None or len(weights) == 0:
        weights = np.ones_like(labels)
    
    # Square both predictions and labels
    squared_preds = preds ** 2
    squared_labels = labels ** 2
    
    # Compute the weighted mean absolute error
    weighted_mae = np.sum(np.abs(squared_preds - squared_labels) * weights) / np.sum(weights)
    
    # Return a tuple: (metric_name, metric_value, is_higher_better)
    return ('mae', weighted_mae)

In [ ]:
%%time

import xgboost as xgb

# Define features (exclude target and date; adjust if you need to drop additional columns)
features = [col for col in X_train.columns if col not in ['date', 'name']]

lr = .1
es = 100
n_est = round(5000/lr)
seed = 2
base_params = {
    'n_estimators':n_est
    ,'learning_rate':lr
    ,'verbosity':0
    ,'enable_categorical':True
    ,'early_stopping_rounds':es
    ,'random_state':seed
    ,'objective':'reg:squarederror'
    ,'eval_metric':'rmse'
    ,'device':'cuda'
    ,'reg_lambda':0
    ,'min_child_weight':1
}

xgb = XGBRegressor(**base_params)
xgb.fit(X_train[features],
        np.sqrt(y_train),
        sample_weight=train_weight,
        eval_set=[(test[features], np.sqrt(test_sales))],
        eval_sample_weight=[test_weight],
        verbose=100)

In [ ]:
%%time
train_preds = xgb.predict(X_train[features])
test_preds = xgb.predict(test[features])

train_mae = mean_absolute_error(train_preds**2, y_train, sample_weight = train_weight)
test_mae = mean_absolute_error(test_preds**2, test_sales, sample_weight = test_weight)
print(train_mae)
print(test_mae)

In [ ]:
ax = xgb.plot_importance(model, max_num_features=10, importance_type='gain')
plt.show()

Per a suggestion from https://datascience.stackexchange.com/questions/104743/optimizing-mae-degrades-mae-metrics, I take the square root of sales and optimize the mean squared error as a sort-of proxy for optimizing mean absolute error. In this case, it's more effective than directly trying to optimize for mean absolute error.

## Looking at model outputs

In [ ]:
concatted = pd.concat([test, test_sales], axis=1)
concatted['error'] = mean_absolute_error(concatted['sales'], concatted['last_sales_ema005'], sample_weight=test_weight)

concatted.sort_values(by='error')

## Shap notes below

I was running into errors when trying to directly apply the shap package functions to the xgboost model, so here's a workaround that I found online.

In [ ]:
test_dm = DMatrix(test_copy, enable_categorical=True)
shap_values = xgb.get_booster().predict(test_dm, pred_contribs=True)
shap.summary_plot(shap_values[:,:-1], test_copy, max_display=40)

I would often look at the shap partial dependence plots for features in the model. This helped uncover some valuable insights for effective feature engineering. For example, one curious thing I noticed was that discount 6 uniquely appears to have a negative correlation with predicted sales. This led me to exclude discount 6 when calculating the max discount. Note that this trend is more apparent before adding max_discount and other discount-based feature engineering to the model).

In [ ]:
shap.dependence_plot('type_6_discount', shap_values[:,:-1], test_copy)

In [ ]:
test_sub = test_pred_df.mean(axis=1)
test_sub.name = 'sales_hat'
test_sub.to_csv('submission.csv')